In [1]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\volpi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\volpi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\volpi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\volpi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
import sys


import re, unicodedata
import contractions
import inflect
import joblib
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.decomposition import PCA, TruncatedSVD
from collections import Counter
from wordcloud import WordCloud
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import seaborn as sns
import joblib


### Carga de datos

In [3]:
datos = pd.read_csv('fake_news_spanish.csv', sep = ';', encoding = 'utf-8')
data = datos.copy()

### Funciones procesamiento de datos

In [4]:
def remove_non_ascii(words):
    new_words = []
    for word in words:
        if word is not None:
          new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
          new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        if word is not None:
            new_word = word.lower()
            new_words.append(new_word)
    return new_words
            
def remove_punctuation(words):
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def replace_numbers(words):
   p = inflect.engine()
   new_words = []
   for word in words:
       if word.isdigit():
           new_word = p.number_to_words(word)
           new_words.append(new_word)
 
       else:
           new_words.append(word)
   return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    stop_words = set(stopwords.words('spanish'))

    for w in words:
        if w not in stop_words:
            new_words.append(w)
    return new_words

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

### Funciones de normalizacion

In [8]:
def stem_words(words):
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
   stems = stem_words(words)
   lemmas = lemmatize_verbs(words)
   return stems + lemmas


### Preprocesamiento + Tokenizacion + Normalizacion

In [10]:
def preprocesamiento_completo(data):
    data.dropna()
    data['Titulo'] = data['Titulo'].fillna(' ')
    data = data.drop_duplicates(subset = ['Titulo', 'Descripcion'], keep = 'first')
    data['Fecha'] = pd.to_datetime(data['Fecha'], errors='coerce')
    data = data.drop(columns='ID')
    data['Descripcion'] = data['Descripcion'].apply(contractions.fix)
    data['Titulo'] = data['Titulo'].apply(contractions.fix)
    data['words_descripcion'] = data['Descripcion'].apply(word_tokenize)
    data['words_titulo'] = data['Titulo'].apply(word_tokenize) 
    data['words_descripcion'].dropna()
    data['words_titulo'].dropna()
    data['prep_descripcion'] = data['words_descripcion'].apply(preprocessing)
    data['prep_titulo'] = data['words_titulo'].apply(preprocessing)
    data['prep_descripcion'] = data['prep_descripcion'].apply(stem_and_lemmatize) 
    data['prep_titulo'] = data['prep_titulo'].apply(stem_and_lemmatize)
    data['prep_descripcion'] = data['prep_descripcion'].apply(lambda x: ' '.join(map(str, x)))
    data['prep_titulo'] = data['prep_titulo'].apply(lambda x: ' '.join(map(str, x)))
    data["concatenado"] = data["prep_titulo"] + " " + data["prep_descripcion"]
    
    return data

    

In [11]:
preprocesamiento_completo(data)

C:\Users\volpi\AppData\Local\Temp\ipykernel_28568\63361374.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Fecha'] = pd.to_datetime(data['Fecha'], errors='coerce')


,Label,Titulo,Descripcion,Fecha,words_descripcion,words_titulo,prep_descripcion,prep_titulo,concatenado
0,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,2023-02-06,"[El, diario, británico, publicó, este, pasado,...","['The, Guardian, ', va, con, Sánchez, :, 'Euro...",diario britanico publico pasado juev edit prox...,the guard va sanchez europ necesit apuest frut...,the guard va sanchez europ necesit apuest frut...
1,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,2023-01-10,"[REVELAN, QUE, EL, GOBIERNO, NEGOCIO, LA, LIBE...","[REVELAN, QUE, EL, GOBIERNO, NEGOCIO, LA, LIBE...",revel gobierno negocio liberac mirel cambio ot...,revel gobierno negocio liberac mirel cambio ot...,revel gobierno negocio liberac mirel cambio ot...
2,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,NaT,"[El, valencianismo, convoca, en, Castelló, su,...","[El, 'Ahora, o, nunca, ', de, Joan, Fuster, so...",valencianismo convoc castello fiest grand conm...,ahor nunc joan fust estatuto valenciano cumpl ...,ahor nunc joan fust estatuto valenciano cumpl ...
3,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",2022-03-01,"[En, política, ,, igual, que, hay, que, negoci...","[Iglesias, alienta, a, Yolanda, Díaz, ,, ERC, ...",politic ig negoci empresario negoci grupo parl...,iglesia alient yoland diaz erc eh bildu negoci...,iglesia alient yoland diaz erc eh bildu negoci...
4,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",2018-09-03,"[En, una, entrevista, en, El, Punt, Avui, ,, e...","[Puigdemont, :, 'No, sería, ninguna, tragedia,...",entrevist punt avu lid jxcat desdramatizado po...,puigdemont ser ningun traged repet elecc puigd...,puigdemont ser ningun traged repet elecc puigd...
...,...,...,...,...,...,...,...,...,...
57058,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,2021-08-06,"[El, gobierno, regional, han, indicado, que, l...","[El, Defensor, del, Pueblo, reclama, a, la, Co...",gobierno reg indicado at dia incluyendo at inm...,defens pueblo reclam comunidad madrid dato dem...,defens pueblo reclam comunidad madrid dato dem...
57059,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,2020-08-09,"[Si, la, higiene, democrática, nos, lleva, a, ...","[El, EQUO, plantea, ceder, la, presidencia, de...",si higy democratic llev exigenc ten pas person...,equo plante ced presidenc comunidad madrid cs ...,equo plante ced presidenc comunidad madrid cs ...
57060,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,2018-12-07,"[El, coordinador, federal, de, IU, asegura, qu...","[Alberto, Garzón, :, 'Que, los, Borbones, son,...",coordinad fed iu asegur monarqu putrefact coor...,alberto garzon borbon ladron hecho historica c...,alberto garzon borbon ladron hecho historica c...
57061,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,NaT,"[Santiago, Abascal, :, Vox, tiene, el, derecho...","[Vox, exige, entrar, en, el, Gobierno, de, Cas...",santiago abasc vox derecho deb form gobierno c...,vox exig entr gobierno castill leon car vicepr...,vox exig entr gobierno castill leon car vicepr...


### Función de Vectorización y Transformación

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

class Transformer_Representacion_Seleccion:
    def __init__(self, count_vectorizer):
        self.count_vectorizer = count_vectorizer
        self.palabras = None
        self.palabras_deseadas = None  

    def fit(self, X, y=None):
        # Ajustar el CountVectorizer y obtener las palabras (características)
        X_transformed = self.count_vectorizer.fit_transform(X)
        self.palabras = self.count_vectorizer.get_feature_names_out()

        # Crear un DataFrame temporal para realizar las selecciones de palabras relevantes
        X_todas_palabras = pd.DataFrame(X_transformed.toarray(), columns=self.palabras)
        
        # Seleccionar las palabras relevantes usando un ciclo tradicional
        self.palabras_deseadas = []
        for nombre in X_todas_palabras.columns:
            # Contar cuántas filas tienen valores distintos de 0 para cada palabra
            dato = (X_todas_palabras[nombre] != 0).sum()
            if dato > 1:
                # Si la palabra aparece en más de una fila, la agregamos a la lista
                self.palabras_deseadas.append(nombre)
        
        return self

    def transform(self, X):
        # Transformar los datos usando CountVectorizer
        X_transformed = self.count_vectorizer.transform(X)
        
        # Crear DataFrame con todas las palabras
        X_todas_palabras = pd.DataFrame(X_transformed.toarray(), columns=self.palabras)
        
        # Seleccionar solo las palabras relevantes
        palabras_a_usar = pd.Index(self.palabras_deseadas).intersection(X_todas_palabras.columns)
        
        # Retornar el DataFrame con las palabras relevantes
        return X_todas_palabras[palabras_a_usar].copy()

### Creación del Pipeline

In [30]:
from sklearn.pipeline import FunctionTransformer


pipeline = Pipeline([
    ('preprocesamiento', FunctionTransformer(preprocesamiento_completo)),
    ('representacion', Transformer_Representacion_Seleccion(CountVectorizer())),
    ('clasificador', LogisticRegression(C=1, max_iter=1000, solver='saga') )
])

In [ ]:

data_preprocessed = preprocesamiento_completo(data)

X = data_preprocessed['concatenado']
y = data_preprocessed['Label']

pipeline.fit(X, y)